## **Turi Object Detection**
Welcome to the object detection model quickstart on Skafos! The purpose of this notebook is to get you going end-to-end. Below we will do the following:

1. Load images of cars, bikes and people and their corresponding [bounding boxes](https://computersciencewiki.org/index.php/Bounding_boxes).
2. Build an object detection model for detecting cars, bikes and people in images.
3. Convert the model to CoreML format and save it to the Skafos framework.

The example is based on Turi Create's Object Detection model.

---

Execute each cell one-by-one, by selecting the cell and do one of the following:

- Clicking the "play" button at the top of this frame.
- Typing 'Control + Enter' or 'Shift + Enter'.

In [ ]:
# If this is your first time in the JupyterLab workspace - install external dependencies
from utilities.dependencies import install
install(timeout=500)

# No need to do this in the future for this notebook

In [ ]:
# Import necessary libraries
import tarfile
import urllib.request

import coremltools
import turicreate as tc
from skafossdk import *

In [ ]:
# Initialize Skafos
ska = Skafos()

### 1. **Load the data**
The data for this example are 1096 images, which breaks down to 420 cars and 365 bikes and 311 people. The data set can be found [here](https://lear.inrialpes.fr/people/marszalek/data/ig02/).  To see how this data was created from the raw data set, check out `advanced_usage/load_object_data.ipynb`. The data is randomly split into train and test sets, where 80% of the data is used for training, and 20% is used for model evaluation.

In [ ]:
# Specify the data set download url
data_url = "https://s3.amazonaws.com/skafos.example.data/ObjectDetection/ig02.sframe.tar.gz"
data_path = "ig02.sframe.tar.gz"

# Pull the compressed data and extract it
retrieve = urllib.request.urlretrieve(data_url, data_path)
tar = tarfile.open(data_path)
tar.extractall()
tar.close()

In [ ]:
## Load the data
data =  tc.SFrame('ig02.sframe')

## Make a train-test split
train_data, test_data = data.random_split(0.8)

### 2. **Build the model**
We pass the data to the `tc.object_detector.create` function and specify a few arguments needed to properly run the model. To understand more about this specific function, check out the [Turi Create Documentation](https://apple.github.io/turicreate/docs/userguide/object_detection/).

**We recommend training this model on a GPU instead of CPU. Additionally, if you don't update the training dataset, it will re-create the baseline object detection model which can be found in the integration guide on the Skafos dashboard.** [You can leverage a GPU by deploying this code as a job](https://docs.metismachine.io/v1.3.1/docs/using-a-gpu).

In [ ]:
# Create a model
model = tc.object_detector.create(
    dataset=train_data,
    max_iterations=0, # if 0, Turi Create determines # of iterations
)

### 3. **Deliver the model**
Once your model has been created, it must be converted to CoreML and saved to the Skafos framework. Once saved, if you wish to push to your iOS devices, you can use the `.deliver()` method below. We've left that commented out for now.

In [ ]:
# Specify the CoreML model name
model_name = 'ObjectDetection'
coreml_model_name = model_name + '.mlmodel'

# Export the trained model to CoreML format
res = model.export_coreml(coreml_model_name) 

# Use coremltools to convert model weights to half-precision.
# This may be necessary if you have memory concerns within your app
#model_spec = coremltools.utils.load_spec(coreml_model_name)
#model_fp16_spec = coremltools.utils.convert_neural_network_spec_weights_to_fp16(model_spec)
#coremltools.utils.save_spec(model_fp16_spec, coreml_model_name)

# Save model asset to Skafos
ska.asset_manager.save(
    name=model_name,              # Name used to load or deliver asset, also used within the Swift SDK.
    files=coreml_model_name,      # File or list of files to bundle together as a versioned asset.
    tags=['latest'],              # User-defined tags to help distinguish your asset.
    access='public'               # Asset access- public/private.
)

In [ ]:
# Deliver asset to devices (push)
#ska.asset_manager.deliver(
#  name=model_name,                # Name used to load or deliver asset, also used within the Swift SDK.
#  tag='latest',                   # User-defined tags to help distinguish your asset.
#  dev=True                        # Push asset through Apple's APNS dev or prod server
#)